# Mahabharata

## Frame

Central Theme: Karna's life shaped due to his actions

Exploratory Questions:
- Other code sample has the questions


## Acquire

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk import FreqDist

In [3]:
with open("../Data/mhbhrt/mahafull.txt", "r") as f:
    file = f.read()

In [4]:
# Get all the chapters
raw = file.split("BOOK")[1:]
len(raw)
# raw[0]

18

In [5]:
# Create the dataframe
mhb_df = pd.DataFrame({"book": raw})


## Refine

Further break down the books to get the following elements in its structure.
- sections
- sentences
- words

### Get chapter & title text

In [6]:
 books = mhb_df.copy()

In [7]:
books['book_title'] = books.book.str.split("\n").str[2]
books['book_num'] = books.index+1
books

,book,book_title,book_num
0,1\n\nADI PARVA\n\nTRANSLATOR'S PREFACE\n\nThe...,ADI PARVA,1
1,2\n\nSABHA PARVA\n\n\nSECTION I\n\n(Sabhakriy...,SABHA PARVA,2
2,3 \n\nVANA PARVA\n\nSECTION I\n\n(Aranyaka Pa...,VANA PARVA,3
3,4\n\nVIRATA PARVA\n\nSECTION I\n\n(Pandava-Pr...,VIRATA PARVA,4
4,5\n\nUDYOGA PARVA\n\nSECTION I\n\nOM! HAVING ...,UDYOGA PARVA,5
5,6\n\nBHISHMA PARVA\n\nSECTION I\n\n(Jamvu-kha...,BHISHMA PARVA,6
6,7\n\nDRONA PARVA\n\nSECTION I\n\n(Dronabhishe...,DRONA PARVA,7
7,8\n\nKARNA-PARVA\n\nSECTION 1\n\nOm! Having b...,KARNA-PARVA,8
8,9\n\nSHALYA-PARVA\n\nSECTION 1\n\nOm! Having ...,SHALYA-PARVA,9
9,10\n\nSAUPTIKA-PARVA\n\nSECTION 1\n\nOm! Havi...,SAUPTIKA-PARVA,10


In [10]:
# book titles and book_setions
book_sections=["book_title" + "\t" + "section_title" + "\t" + "section_content"]
for j in range(len(raw)):
    book_title = raw[j].split("\n")[2]
    book_section = raw[j].split("SECTION")[1:]
    for i in range(len(book_section)):
        loop_var = book_title + '/t' + str(book_section[i].split("\n")[0].strip()) +'\t'+ str(book_section[i][1:])
        book_sections.append(loop_var)
book_sections[2]

'ADI PARVA/tII\tII\n\n"The Rishis said, \'O son of Suta, we wish to hear a full and\ncircumstantial account of the place mentioned by you as Samanta-panchaya.\'\n\n"Sauti said, \'Listen, O ye Brahmanas, to the sacred descriptions I utter\nO ye best of men, ye deserve to hear of the place known as\nSamanta-panchaka. In the interval between the Treta and Dwapara Yugas,\nRama (the son of Jamadagni) great among all who have borne arms, urged by\nimpatience of wrongs, repeatedly smote the noble race of Kshatriyas. And\nwhen that fiery meteor, by his own valour, annihilated the entire tribe\nof the Kshatriyas, he formed at Samanta-panchaka five lakes of blood. We\nare told that his reason being overpowered by anger he offered oblations\nof blood to the manes of his ancestors, standing in the midst of the\nsanguine waters of those lakes. It was then that his forefathers of whom\nRichika was the first having arrived there addressed him thus, \'O Rama, O\nblessed Rama, O offspring of Bhrigu, we

### Generic Unnest Function

In [11]:
def unnest_tokens(dataframe, column, tokenizer, new_column='word'): 
    """
    Applies a tokenizer to a column and then unnest the dataframe
    
    dataframe: dataframe with text
    column: name of the column with the text
    new_column: what you want the column of words to be called
    tokenizer: a function to be used to tokenize the text
    """ 
    new_column_num = new_column + "_num"
    df = ( dataframe[column]
              .apply(tokenizer)
              .apply(pd.Series)
              .stack()
              .reset_index(level=1)
              .rename(columns={0: new_column, "level_1":new_column_num})
              .join(dataframe.drop(column, axis=1), how='left')
              .reset_index(drop=True)
         )
    df[new_column_num] = pd.to_numeric(df[new_column_num])
    df[new_column_num] = df[new_column_num] + 1
    return df

### Section-Tokenzier

In [12]:
def section_tokenizer(text):
    return text.split("SECTION")[1:]

In [13]:
#splitting into sections using the unnest function
sections = unnest_tokens(books, "book", section_tokenizer, "section")
sections.head()

,section_num,section,book_title,book_num
0,1,I\n\nOm! Having bowed down to Narayana and Na...,ADI PARVA,1
1,2,"II\n\n""The Rishis said, 'O son of Suta, we wi...",ADI PARVA,1
2,3,"III\n\n(Paushya Parva)\n\nSauti said, ""Janame...",ADI PARVA,1
3,4,"IV\n\n(Pauloma Parva)\n\n'UGRASRAVA SAUTI, th...",ADI PARVA,1
4,5,"V\n\n(Pauloma Parva continued)\n\n""Saunaka sa...",ADI PARVA,1


In [14]:
sections['section'][1]

' II\n\n"The Rishis said, \'O son of Suta, we wish to hear a full and\ncircumstantial account of the place mentioned by you as Samanta-panchaya.\'\n\n"Sauti said, \'Listen, O ye Brahmanas, to the sacred descriptions I utter\nO ye best of men, ye deserve to hear of the place known as\nSamanta-panchaka. In the interval between the Treta and Dwapara Yugas,\nRama (the son of Jamadagni) great among all who have borne arms, urged by\nimpatience of wrongs, repeatedly smote the noble race of Kshatriyas. And\nwhen that fiery meteor, by his own valour, annihilated the entire tribe\nof the Kshatriyas, he formed at Samanta-panchaka five lakes of blood. We\nare told that his reason being overpowered by anger he offered oblations\nof blood to the manes of his ancestors, standing in the midst of the\nsanguine waters of those lakes. It was then that his forefathers of whom\nRichika was the first having arrived there addressed him thus, \'O Rama, O\nblessed Rama, O offspring of Bhrigu, we have been gra

In [15]:
# How many in each book
(sections.groupby("book_num").section.count())

book_num
1     236
2      80
3     313
4      72
5     199
6     124
7     200
8      96
9      65
10     18
11     27
12    363
13    168
14     92
15     39
16      8
17      3
18      6
Name: section, dtype: int64

In [16]:
# Replacing the \n with a space character.
sections['section'] = sections.section.str.replace("\n", " ")
sections.section[1]

' II  "The Rishis said, \'O son of Suta, we wish to hear a full and circumstantial account of the place mentioned by you as Samanta-panchaya.\'  "Sauti said, \'Listen, O ye Brahmanas, to the sacred descriptions I utter O ye best of men, ye deserve to hear of the place known as Samanta-panchaka. In the interval between the Treta and Dwapara Yugas, Rama (the son of Jamadagni) great among all who have borne arms, urged by impatience of wrongs, repeatedly smote the noble race of Kshatriyas. And when that fiery meteor, by his own valour, annihilated the entire tribe of the Kshatriyas, he formed at Samanta-panchaka five lakes of blood. We are told that his reason being overpowered by anger he offered oblations of blood to the manes of his ancestors, standing in the midst of the sanguine waters of those lakes. It was then that his forefathers of whom Richika was the first having arrived there addressed him thus, \'O Rama, O blessed Rama, O offspring of Bhrigu, we have been gratified with the 

### Get Sentences

In [17]:
import spacy
nlp = spacy.load('en')

In [18]:
def sentence_tokenizer(text):
    doc = nlp(text)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [19]:
sentences = unnest_tokens(sections, "section", sentence_tokenizer, "sentence")

In [20]:
sentences.head()

,sentence_num,sentence,section_num,book_title,book_num
0,1,I Om!,1,ADI PARVA,1
1,2,"Having bowed down to Narayana and Nara, the mo...",1,ADI PARVA,1
2,3,"Ugrasrava, the son of Lomaharshana, surnamed S...",1,ADI PARVA,1
3,4,"Those ascetics, wishing to hear his wonderful ...",1,ADI PARVA,1
4,5,Having been entertained with due respect by th...,1,ADI PARVA,1


In [21]:
sentences.shape

(136923, 5)

In [22]:
sentences.sentence[1]

'Having bowed down to Narayana and Nara, the most exalted male being, and also to the goddess Saraswati, must the word Jaya be uttered.'

### Get words

In [23]:
test = sentences.sentence[0]
doc = nlp(test)
doc

I  Om!

In [25]:
def word_tokenizer(text):
    doc = nlp(text)
    words = [[token.text, token.lemma_, token.pos_, token.is_stop] for token in doc]
    return words

In [26]:
words = unnest_tokens(sentences, "sentence", word_tokenizer, "word")

In [30]:
words[['word', 'lemma', 'POS', "stop"]] = pd.DataFrame(words.word.values.tolist())

In [31]:
words.head()

,word_num,word,sentence_num,section_num,book_title,book_num,lemma,POS,stop
0,1,I,1,1,ADI PARVA,1,-PRON-,PRON,False
1,2,,1,1,ADI PARVA,1,,SPACE,False
2,3,Om,1,1,ADI PARVA,1,om,PROPN,False
3,4,!,1,1,ADI PARVA,1,!,PUNCT,False
4,1,Having,2,1,ADI PARVA,1,have,VERB,False


In [43]:
words_book1 = pd.Series(words.loc[words['book_num'] == 18].index)
len(words_book1)
words_book1

0        3049888
1        3049889
2        3049890
3        3049891
4        3049892
5        3049893
6        3049894
7        3049895
8        3049896
9        3049897
10       3049898
11       3049899
12       3049900
13       3049901
14       3049902
15       3049903
16       3049904
17       3049905
18       3049906
19       3049907
20       3049908
21       3049909
22       3049910
23       3049911
24       3049912
25       3049913
26       3049914
27       3049915
28       3049916
29       3049917
          ...   
10322    3060210
10323    3060211
10324    3060212
10325    3060213
10326    3060214
10327    3060215
10328    3060216
10329    3060217
10330    3060218
10331    3060219
10332    3060220
10333    3060221
10334    3060222
10335    3060223
10336    3060224
10337    3060225
10338    3060226
10339    3060227
10340    3060228
10341    3060229
10342    3060230
10343    3060231
10344    3060232
10345    3060233
10346    3060234
10347    3060235
10348    3060236
10349    30602

### Deriving the adjectives used for a characters

In [33]:
# Mentions of the Charachters
krn_mention = pd.Series(words.loc[words['word'] == 'Karna'].index)
arj_mention = pd.Series(words.loc[words['word'] == 'Arjuna'].index)
words.loc[2]['POS'] == 'ADJ'
# krn_mention

False

In [47]:
# Deriving the adjectives around Karna
krn_temp = pd.DataFrame()
for t in range(len(krn_mention)):
    i = krn_mention[t] - 5
    j = krn_mention[t] - 5
    while True:
        if((words.loc[j]['POS'] == 'ADJ') & (words.loc[j]['lemma'] != '-PRON-')):
            hrk = words.loc[j]
            krn_temp = krn_temp.append(hrk)
            j=j+1
        else:
            j=j+1
        if(j > i+10):
            break
        
krn_temp

,POS,book_num,book_title,lemma,section_num,sentence_num,stop,word,word_num
2688,ADJ,1.0,ADI PARVA,fair,1.0,78.0,0.0,fair,87.0
6614,ADJ,1.0,ADI PARVA,same,1.0,185.0,1.0,same,23.0
6615,ADJ,1.0,ADI PARVA,which,1.0,185.0,1.0,which,24.0
6805,ADJ,1.0,ADI PARVA,brave,1.0,192.0,0.0,brave,7.0
6808,ADJ,1.0,ADI PARVA,invincible,1.0,192.0,0.0,invincible,10.0
10843,ADJ,1.0,ADI PARVA,souled,2.0,82.0,0.0,souled,14.0
12999,ADJ,1.0,ADI PARVA,incited,2.0,136.0,0.0,incited,112.0
14277,ADJ,1.0,ADI PARVA,gratified,2.0,152.0,0.0,gratified,29.0
15109,ADJ,1.0,ADI PARVA,evil,2.0,177.0,0.0,evil,19.0
15138,ADJ,1.0,ADI PARVA,same,2.0,177.0,1.0,same,48.0


In [45]:
# Deriving the characters around Karna
krn_assoc = pd.DataFrame()
for t in range(len(krn_mention)):
    i = krn_mention[t] - 5
    j = krn_mention[t] - 5
    while True:
        if(words.loc[j]['POS'] == 'PROPN'):
            hrk = words.loc[j]
            krn_assoc = krn_assoc.append(hrk)
            j=j+1
        else:
            j=j+1
        if(j > i+10):
            break
        
krn_assoc

,POS,book_num,book_title,lemma,section_num,sentence_num,stop,word,word_num
2685,PROPN,1.0,ADI PARVA,karna,1.0,78.0,0.0,Karna,84.0
5250,PROPN,1.0,ADI PARVA,karna,1.0,148.0,0.0,Karna,13.0
6463,PROPN,1.0,ADI PARVA,karna,1.0,182.0,0.0,Karna,5.0
6512,PROPN,1.0,ADI PARVA,kritavarma,1.0,183.0,0.0,Kritavarma,7.0
6514,PROPN,1.0,ADI PARVA,kripa,1.0,183.0,0.0,Kripa,9.0
6516,PROPN,1.0,ADI PARVA,karna,1.0,183.0,0.0,Karna,11.0
6521,PROPN,1.0,ADI PARVA,drona,1.0,183.0,0.0,Drona,16.0
6559,PROPN,1.0,ADI PARVA,indra,1.0,184.0,0.0,Indra,10.0
6562,PROPN,1.0,ADI PARVA,karna,1.0,184.0,0.0,Karna,13.0
6566,PROPN,1.0,ADI PARVA,madhava,1.0,184.0,0.0,Madhava,17.0


In [48]:
krn_assoc.to_csv('krn_assoc.csv')

In [40]:
krn_temp.to_csv('krn_adj.csv')

In [35]:
# Seeing the most used adjectives around Karna
adj_krn = pd.Series(krn_temp['word'])
adj_krn
fdistk=FreqDist(adj_krn)
fdistk.most_common(30)
# krn_temp.lemma.unique()

[('mighty', 117),
 ('great', 96),
 ('that', 57),
 ('thy', 54),
 ('all', 44),
 ('heroic', 30),
 ('souled', 29),
 ('other', 27),
 ('many', 26),
 ('which', 25),
 ('wicked', 21),
 ('desirous', 19),
 ('armed', 19),
 ('very', 19),
 ('fierce', 16),
 ('keen', 16),
 ('loud', 15),
 ('single', 14),
 ('own', 12),
 ('invincible', 11),
 ('latter', 11),
 ('terrible', 10),
 ('beautiful', 9),
 ('equal', 9),
 ('straight', 9),
 ('such', 9),
 ('superior', 9),
 ('brave', 8),
 ('harsh', 8),
 ('large', 8)]

In [41]:
# Deriving the adjectives around Arjuna
arj_temp = pd.DataFrame()
for t in range(len(arj_mention)):
    i = arj_mention[t] - 5
    j = arj_mention[t] - 5
    while True:
        if((words.loc[j]['POS'] == 'ADJ') & (words.loc[j]['lemma'] != '-PRON-')):
            hrk = words.loc[j]
            arj_temp = arj_temp.append(hrk)
            j=j+1
        else:
            j=j+1
        if(j > i+10):
            break
        
arj_temp

,POS,book_num,book_title,lemma,section_num,sentence_num,stop,word,word_num
3729,ADJ,1.0,ADI PARVA,submissive,1.0,109.0,0.0,submissive,18.0
3765,ADJ,1.0,ADI PARVA,virgin,1.0,110.0,0.0,virgin,8.0
4523,ADJ,1.0,ADI PARVA,forcible,1.0,132.0,0.0,forcible,14.0
4583,ADJ,1.0,ADI PARVA,celestial,1.0,133.0,0.0,celestial,9.0
5082,ADJ,1.0,ADI PARVA,just,1.0,144.0,1.0,just,6.0
5084,ADJ,1.0,ADI PARVA,renowned,1.0,144.0,0.0,renowned,8.0
5269,ADJ,1.0,ADI PARVA,free,1.0,148.0,0.0,free,32.0
5375,ADJ,1.0,ADI PARVA,noble,1.0,151.0,0.0,noble,17.0
5379,ADJ,1.0,ADI PARVA,single,1.0,151.0,0.0,single,21.0
5455,ADJ,1.0,ADI PARVA,virtuous,1.0,153.0,0.0,virtuous,13.0


In [42]:
arj_temp.to_csv('arj_adj.csv')

In [49]:
# Deriving the characters around Karna
arj_assoc = pd.DataFrame()
for t in range(len(arj_mention)):
    i = arj_mention[t] - 5
    j = arj_mention[t] - 5
    while True:
        if(words.loc[j]['POS'] == 'PROPN'):
            hrk = words.loc[j]
            arj_assoc = arj_assoc.append(hrk)
            j=j+1
        else:
            j=j+1
        if(j > i+10):
            break
        
arj_assoc

,POS,book_num,book_title,lemma,section_num,sentence_num,stop,word,word_num
3262,PROPN,1.0,ADI PARVA,arjuna,1.0,93.0,0.0,Arjuna,13.0
3267,PROPN,1.0,ADI PARVA,bhimasena,1.0,93.0,0.0,Bhimasena,18.0
3721,PROPN,1.0,ADI PARVA,yudhishthira,1.0,109.0,0.0,Yudhishthira,10.0
3726,PROPN,1.0,ADI PARVA,arjuna,1.0,109.0,0.0,Arjuna,15.0
3762,PROPN,1.0,ADI PARVA,arjuna,1.0,110.0,0.0,Arjuna,5.0
3766,PROPN,1.0,ADI PARVA,krishna,1.0,110.0,0.0,Krishna,9.0
3876,PROPN,1.0,ADI PARVA,bhimasena,1.0,113.0,0.0,Bhimasena,17.0
3878,PROPN,1.0,ADI PARVA,arjuna,1.0,113.0,0.0,Arjuna,19.0
3881,PROPN,1.0,ADI PARVA,jarasandha,1.0,113.0,0.0,Jarasandha,22.0
4462,PROPN,1.0,ADI PARVA,arjuna,1.0,131.0,0.0,Arjuna,5.0


In [50]:
arj_assoc.to_csv('arj_assoc.csv')

In [260]:
# Seeing the most used adjectives around Arjuna
adj_arj = pd.Series(arj_temp['word'])
adj_arj
fdistk=FreqDist(adj_arj)
fdistk.most_common(30)
# krn_temp.lemma.unique()

[('great', 88),
 ('mighty', 83),
 ('that', 74),
 ('thy', 58),
 ('white', 43),
 ('all', 42),
 ('other', 33),
 ('which', 32),
 ('armed', 26),
 ('heroic', 25),
 ('many', 22),
 ('fierce', 22),
 ('own', 21),
 ('hostile', 16),
 ('desirous', 16),
 ('illustrious', 15),
 ('equal', 14),
 ('loud', 14),
 ('same', 13),
 ('latter', 13),
 ('very', 13),
 ('incapable', 12),
 ('whose', 12),
 ('superior', 12),
 ('keen', 12),
 ('best', 11),
 ('silent', 11),
 ('victorious', 11),
 ('able', 11),
 ('single', 10)]